In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd 
import numpy as np
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score,classification,roc_curve
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV


In [0]:
data = pd.read_csv("/content/gdrive/My Drive/ZS/final_2.csv")  

In [0]:
data.isnull().sum()

In [0]:
data1 = data.copy()

In [0]:
data1 = data1.drop(['type_of_combined_shot','shot_id_number','match_event_id','team_id'],axis=1)

In [0]:
data1.isnull().sum()

In [0]:
mat_id=data1['match_id'].unique()

In [0]:
gb = data1.groupby('match_id') 

In [0]:
for i in range(len(mat_id)):
  if pd.isnull(gb.get_group(mat_id[i])['home/away'].reset_index().drop(['index'],1).iloc[:,0][0]) and len(gb.get_group(mat_id[i])['home/away'].reset_index().drop(['index'],1).iloc[:,0]) > 1:
    x=gb.get_group(mat_id[i])['home/away'].reset_index().drop(['index'],1).iloc[:,0][1]
  else:
    x=gb.get_group(mat_id[i])['home/away'].reset_index().drop(['index'],1).iloc[:,0][0]
  if (pd.isnull(x)==False):
    data1.loc[data1['match_id']==mat_id[i],['home/away']]=data1.groupby('match_id').get_group(mat_id[i])['home/away'].fillna(x)
  
  if pd.isnull(gb.get_group(mat_id[i])['date_of_game'].reset_index().drop(['index'],1).iloc[:,0][0]) and len(gb.get_group(mat_id[i])['date_of_game'].reset_index().drop(['index'],1).iloc[:,0]) > 1:
    y=gb.get_group(mat_id[i])['date_of_game'].reset_index().drop(['index'],1).iloc[:,0][1]
  else:
    y=gb.get_group(mat_id[i])['date_of_game'].reset_index().drop(['index'],1).iloc[:,0][0]
  if (pd.isnull(y)==False):
    data1.loc[data1['match_id']==mat_id[i],['date_of_game']]=data1.groupby('match_id').get_group(mat_id[i])['date_of_game'].fillna(y)
  
  if pd.isnull(gb.get_group(mat_id[i])['lat/lng'].reset_index().drop(['index'],1).iloc[:,0][0]) and len(gb.get_group(mat_id[i])['lat/lng'].reset_index().drop(['index'],1).iloc[:,0]) > 1:
    z=gb.get_group(mat_id[i])['lat/lng'].reset_index().drop(['index'],1).iloc[:,0][1]
  else:
    z=gb.get_group(mat_id[i])['lat/lng'].reset_index().drop(['index'],1).iloc[:,0][0]
  if (pd.isnull(z)==False):
    data1.loc[data1['match_id']==mat_id[i],['lat/lng']]=data1.groupby('match_id').get_group(mat_id[i])['lat/lng'].fillna(z)

In [0]:
data1.isnull().sum()

In [0]:
data1.shape

In [0]:
def add1(x):
  y=x+1
  x=str(x)
  y=str(y)
  y=y[2:]
  x=x+"-"+y
  return(x) 

In [0]:
def sub1(x):
  y=x-1
  x=str(x)
  y=str(y)
  x=x[2:]
  y=y+"-"+x
  return(y)

In [0]:
for i in range(len(data1['match_id'])):
  if pd.isnull(data1['date_of_game'][i])==False:
    mon = data1['date_of_game'][i].split('-')[1]
    year = data1['date_of_game'][i].split('-')[2]
    if int(mon)>=6:
      data1['game_season'][i] = add1(int(year))
    else:
      data1['game_season'][i] = sub1(int(year))
      
    

In [0]:
data1.isnull().sum()

In [0]:
data2 = data1.copy()

In [0]:
min1 = data1['remaining_min.1'].unique()

In [0]:
min1=min1.tolist()

In [0]:
min1.sort()

In [0]:
l = [0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0]

In [0]:
for i in l:
  min1.remove(i)

In [0]:
for i in range(len(min1)):
  if np.isnan(min1[i])==False:
    print(i)
    x=data1.loc[data1['remaining_min.1']==min1[i],['remaining_min']].mode()
    if len(x)>0:
      data1.loc[data1['remaining_min.1']==min1[i],['remaining_min']]=data1.loc[data1['remaining_min.1']==min1[i],['remaining_min']].fillna(x.iloc[0,:][0])

In [188]:
data1['remaining_min'].unique()

array([10.,  7.,  6.,  0.,  9.,  8.,  3.,  1., 11.,  2.,  5.,  4., nan])

In [0]:
data1.isnull().sum()

In [0]:
min1 = data1['power_of_shot.1'].unique()

In [0]:
min1=min1.tolist()
min1.sort()

In [0]:
min1

In [0]:
l = [1.0,2.0,3.0,4.0,5.0,6.0,7.0]

In [0]:
for i in l:
  min1.remove(i)

In [0]:
for i in range(len(min1)):
  if np.isnan(min1[i])==False:
    print(i)
    x=data1.loc[data1['power_of_shot.1']==min1[i],['power_of_shot']].mode()
    if len(x)>0:
      data1.loc[data1['power_of_shot.1']==min1[i],['power_of_shot']]=data1.loc[data1['power_of_shot.1']==min1[i],['power_of_shot']].fillna(x.iloc[0,:][0])

In [0]:
data1['power_of_shot'].unique()

In [0]:
data1.isnull().sum()

In [0]:
min1 = data1['knockout_match.1'].unique()
min1=min1.tolist()
min1.sort()
min1

In [0]:
l = [0.0,1.0]

In [0]:
for i in l:
  min1.remove(i)

In [0]:
for i in range(len(min1)):
  if np.isnan(min1[i])==False:
    x=data1.loc[data1['knockout_match.1']==min1[i],['knockout_match']].mode()
    if len(x)>0:
      data1.loc[data1['knockout_match.1']==min1[i],['knockout_match']]=data1.loc[data1['knockout_match.1']==min1[i],['knockout_match']].fillna(x.iloc[0,:][0])

In [0]:
data1['knockout_match'].unique()

In [0]:
data1.isnull().sum()

In [0]:
min1 = data1['remaining_sec.1'].unique()
min1=min1.tolist()
min1.sort()
min1

In [0]:
z=[]
l = range(60)
for i in l:
  z.append(float(i))

In [0]:
for i in z:
  min1.remove(i)

In [0]:
for i in range(len(min1)):
  if np.isnan(min1[i])==False:
    x=data1.loc[data1['remaining_sec.1']==min1[i],['remaining_sec']].mode()
    if len(x)>0:
      data1.loc[data1['remaining_sec.1']==min1[i],['remaining_sec']]=data1.loc[data1['remaining_sec.1']==min1[i],['remaining_sec']].fillna(x.iloc[0,:][0])

In [0]:
data1['remaining_sec'].unique()

In [0]:
data1.isnull().sum()

In [0]:
min1 = data1['distance_of_shot.1'].unique()
min1=min1.tolist()

In [0]:
z=[]
l = [20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,87,88,89,90,91,94,97,99]
for i in l:
  z.append(float(i))

In [0]:
z[24]

In [0]:
for i in range(len(z)):
  print(z[i],i)
  min1.remove(z[i])

In [0]:
min1.remove(z[24])

In [0]:
for i in range(25,len(z)):
  print(z[i],i)
  min1.remove(z[i])

In [0]:
min1.remove(z[47])

In [0]:
for i in range(48,len(z)):
  print(z[i],i)
  min1.remove(z[i])

In [0]:
min1.remove(z[66])

In [0]:
for i in range(67,len(z)):
  print(z[i],i)
  min1.remove(z[i])

In [0]:
for i in range(len(min1)):
  if np.isnan(min1[i])==False:
    x=data1.loc[data1['distance_of_shot.1']==min1[i],['distance_of_shot']].mode()
    if len(x)>0:
      data1.loc[data1['distance_of_shot.1']==min1[i],['distance_of_shot']]=data1.loc[data1['distance_of_shot.1']==min1[i],['distance_of_shot']].fillna(x.iloc[0,:][0])

In [0]:
data1.isnull().sum()

In [0]:
data1=data1.drop(['distance_of_shot.1','remaining_sec.1','knockout_match.1','power_of_shot.1','remaining_min.1'],axis=1)

In [0]:
data1.shape

In [0]:
data1.isnull().sum()

In [0]:
data1 = data1.drop(['location_x','location_y'],axis=1)

In [0]:
data1.isnull().sum()

In [0]:
def replace1(x):
  data1[x]=data1[x].fillna(int(data1[x].mode()))

In [0]:
replace1("remaining_min")

In [0]:
replace1("power_of_shot")
replace1("knockout_match")
replace1("remaining_sec")
replace1("distance_of_shot")

In [0]:
data1.isnull().sum()

In [0]:
data1['game_season']=data1['game_season'].fillna(data1.mode()['game_season'][0])

In [0]:
data1['date_of_game']=data1['date_of_game'].fillna(data1.mode()['date_of_game'][0])

In [0]:
data1['home/away']=data1['home/away'].fillna(data1.mode()['home/away'][0])

In [0]:
data1['lat/lng']=data1['lat/lng'].fillna(data1.mode()['lat/lng'][0])

In [0]:
data1.isnull().sum()

In [0]:
lat = []
long=[]
for i in range(len(data1['lat/lng'])):
  x=data1['lat/lng'][i].replace(" ", "").split(',')[0]
  y=data1['lat/lng'][i].replace(" ", "").split(',')[1]
  if x.find('-',0)==-1:
    lat.append(float(x))
  else:
    x=data1['lat/lng'][i].replace(" ", "").replace("-", "").split(',')[0]
    lat.append(float(x))
  if y.find('-',0)==-1:
    long.append(float(y))
  else:
    y=data1['lat/lng'][i].replace(" ", "").replace("-", "").split(',')[1]
    long.append(float(y)*-1)  

In [0]:
lat[0],long[0]

In [0]:
data1['home/away']=data1['home/away'].str.replace("@","vs.")

In [0]:
date=[]
month=[]
year =[]
for i in range(len(data1['date_of_game'])):
  date.append(int(data1['date_of_game'][i].split('-')[0]))
  month.append(int(data1['date_of_game'][i].split('-')[1]))
  year.append(int(data1['date_of_game'][i].split('-')[2]))

In [0]:
data1['lat']=lat
data1['long']=long
data1['date']=date
data1['month']=month
data1['year']=year

In [579]:
data1.shape

(30697, 22)

In [0]:
data1.isnull().sum()

In [0]:
data2 = data1.drop(['team_name','date_of_game','lat/lng','match_id'],axis=1)

In [0]:
data2.isnull().sum()

In [0]:
train_data = data2.loc[data2['is_goal'].isnull()!=True,:]

In [0]:
test_data = data2.loc[data2['is_goal'].isnull()==True,:]

In [0]:
train_data.shape,test_data.shape

In [0]:
train_data=train_data.reset_index().drop(['index'],axis=1)

In [0]:
test_data=test_data.reset_index().drop(['index'],axis=1)

In [0]:
y=train_data['is_goal']

In [0]:
X = train_data.drop(['is_goal','Unnamed: 0'],1)

In [0]:
scaler=StandardScaler()

In [0]:
numerical_data=X.select_dtypes(include=np.number)
cols = numerical_data.columns
numerical_data=scaler.fit_transform(numerical_data)
numerical_data = pd.DataFrame(numerical_data,columns=cols)

In [0]:
numerical_data

In [0]:
categorical_data = X.select_dtypes(include=np.object)

In [0]:
categorical_data = pd.get_dummies(categorical_data)

In [0]:
X = pd.concat([numerical_data,categorical_data],axis=1)

In [0]:
X

In [0]:
X.shape

In [0]:
X_train, X_test, y_train, y_test = tts(X,y,test_size=0.3,random_state=0)
X_train.shape, X_test.shape

In [0]:
logit = LogisticRegression()
logit.fit(X_train,y_train)

In [0]:
pred = logit.predict(X_test)

In [0]:
pred

In [0]:
prob = logit.predict_proba(X_test)

In [0]:
prob

In [0]:
confusion_matrix(y_test,pred)

In [0]:
accuracy_score(y_test,pred)

In [0]:
pca = PCA(0.99)
principalComponents = pca.fit_transform(X)

In [0]:
X_train, X_test, y_train, y_test = tts(principalComponents,y,test_size=0.3,random_state=0)
X_train.shape, X_test.shape

In [0]:
clf = SVC(kernel='linear')

In [0]:
clf.fit(X_train,y_train)

In [0]:
pred = clf.predict(X_test)
confusion_matrix(y_test,pred)
accuracy_score(y_test,pred)

In [0]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 60, kernel_initializer = 'uniform', activation = 'relu', input_dim = 99))
# classifier.add(Dropout(p = 0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 60, kernel_initializer = 'uniform', activation = 'relu'))
# classifier.add(Dropout(p = 0.1))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set


In [0]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

In [0]:
y_pred = classifier.predict(X_test)

In [0]:
pred=[]
for i in range(len(y_pred)):
  if y_pred[i]>=0.5:
    pred.append(1)
  else:
    pred.append(0)

In [0]:
accuracy_score(y_test,pred)

In [0]:
confusion_matrix(y_test,pred)

In [0]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 60, kernel_initializer = 'uniform', activation = 'relu', input_dim = 99))
    classifier.add(Dense(units = 60, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [16,25, 32],
              'epochs': [100,200,300,500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,n_jobs = -1)


In [0]:
grid_search = grid_search.fit(X_train, y_train)

In [0]:
best_accuracy = grid_search.best_score_

In [0]:
def make_submission(prediction, sub_name):
  my_submission = pd.DataFrame({'shot_id_number':pd.read_csv('test.csv').Id,'is_goal':prediction})
  my_submission.to_csv('{}.csv'.format(sub_name),index=False)
  print('A submission file has been made')

predictions = NN_model.predict(test)
make_submission(pred,'submission(NN).csv')